In [ ]:
import pandas as pd

# =========================================
# 0. 경로 설정
# =========================================
DATA_PATH = "./../cohort/cohort_ver151_reorder_col.csv"          # 입력 파일
OUT_PATH  = "./cohort_ver151_with_event_name.csv"      # 출력 파일

# =========================================
# 1. event_id -> event_name 매핑 정의
#    (그림에 나온 표 그대로)
# =========================================
EVENT_ID_TO_NAME = {
    1:  "ED_ARRIVAL",
    2:  "ECG_STEMI_FLAG",
    3:  "ECG_TAKEN",
    4:  "TROP_POSITIVE",
    5:  "TROP_TAKEN",
    6:  "ANTI_PLT_ADMIN",
    7:  "ANTI_PLT_ORDER",
    8:  "PCI_START",
    9:  "ED_DEPARTURE",
    10: "DEATH",
    11: "DISCHARGE",
    12: "ICU_INTIME",
    13: "ICU_OUTTIME",
    14: "EOS"  # EOS (End of Seq)
}

# =========================================
# 2. 데이터 로딩
# =========================================
print(f"[LOAD] {DATA_PATH} 로딩 중...")
df = pd.read_csv(DATA_PATH, low_memory=False)
print("[LOAD] 로딩 완료")
print(" - shape :", df.shape)
print(" - columns 예시:", df.columns.tolist()[:10])

# =========================================
# 3. 매핑 적용용 헬퍼 함수
# =========================================
def add_event_name_column(df, id_col, name_col, mapping):
    """
    id_col: 숫자 event_id가 들어 있는 컬럼 이름
    name_col: 새로 만들 event_name 컬럼 이름
    mapping: dict (event_id -> event_name)
    """
    if id_col not in df.columns:
        print(f"[SKIP] '{id_col}' 컬럼이 없어 '{name_col}'는 생성하지 않음")
        return df

    # int가 아닌 경우를 대비해 한 번 정수로 캐스팅(에러는 무시)
    df[id_col] = pd.to_numeric(df[id_col], errors="coerce").astype("Int64")

    unknown_ids = sorted(set(df[id_col].dropna()) - set(mapping.keys()))
    if unknown_ids:
        print(f"[WARN] 매핑에 없는 event_id 발견: {unknown_ids}")

    df[name_col] = df[id_col].map(mapping)
    print(f"[MAKE] '{name_col}' 컬럼 생성 완료 (from '{id_col}')")
    return df

# =========================================
# 4. 각 event_id 컬럼에 대해 이름 컬럼 추가
#    (있을 때만 생성)
# =========================================
df = add_event_name_column(df, "current_event_id", "current_event_name", EVENT_ID_TO_NAME)
df = add_event_name_column(df, "next_event_id",    "next_event_name",    EVENT_ID_TO_NAME)
df = add_event_name_column(df, "target_next_evt",  "target_next_evt_name", EVENT_ID_TO_NAME)

# Disco / Petri-net용으로 별도 event log만 추출하고 싶다면 예시:
# (hadm_id = case id, current_event_name = activity, current_time = timestamp 가정)
if {"hadm_id", "current_event_name"}.issubset(df.columns):
    # 시간 컬럼 후보: current_time이 있으면 사용, 없으면 time_since_start_min으로 대체
    if "current_time" in df.columns:
        time_col = "current_time"
    elif "time_since_start_min" in df.columns:
        time_col = "time_since_start_min"
    else:
        time_col = None

    if time_col is not None:
        event_log_cols = ["hadm_id", "current_event_name", time_col]
        event_log = df[event_log_cols].copy()
        event_log = event_log.rename(
            columns={
                "hadm_id": "case_id",
                "current_event_name": "activity",
                time_col: "timestamp"
            }
        )
        EVENT_LOG_OUT = "./../cohort/cohort_ver151_event_log_for_disco.csv"
        event_log.to_csv(EVENT_LOG_OUT, index=False)
        print(f"[SAVE] Disco용 event log 저장 완료: {EVENT_LOG_OUT}")
    else:
        print("[INFO] timestamp로 쓸 컬럼이 없어 Disco용 event log는 만들지 않음")

# =========================================
# 5. 전체 데이터 저장
# =========================================
df.to_csv(OUT_PATH, index=False)
print(f"[SAVE] event_name 컬럼이 포함된 전체 데이터 저장 완료: {OUT_PATH}")


[LOAD] ./../cohort/cohort_ver151_reorder_col.csv 로딩 중...
[LOAD] 로딩 완료
 - shape : (40817, 27)
 - columns 예시: ['subject_id', 'hadm_id', 'age', 'gender', 'race', 'arrival_transport', 'prefix_len', 'current_event_id', 'time_since_start_min', 'time_since_ed']
[MAKE] 'current_event_name' 컬럼 생성 완료 (from 'current_event_id')
[SKIP] 'next_event_id' 컬럼이 없어 'next_event_name'는 생성하지 않음
[MAKE] 'target_next_evt_name' 컬럼 생성 완료 (from 'target_next_evt')
[SAVE] Disco용 event log 저장 완료: ./../cohort/cohort_ver151_event_log_for_disco.csv
[SAVE] event_name 컬럼이 포함된 전체 데이터 저장 완료: ./../cohort/cohort_ver151_with_event_name.csv
